[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/dgunning/edgartools/blob/main/notebooks/executive-compensation-sec-python.ipynb)

# Executive Compensation from SEC EDGAR Proxy Statements with Python -- Free, No API Key

Use **edgartools** to extract executive compensation data from SEC proxy statements (DEF 14A) -- completely free, no API key or paid subscription required. Every public company must disclose CEO pay, pay-vs-performance, and shareholder returns in their annual proxy filing.

**What you'll learn:**
- Get CEO compensation from any company's proxy statement
- View 5-year executive compensation history
- Analyze pay-vs-performance (CEO pay vs shareholder returns)
- Compare CEO compensation across companies
- Export compensation data to pandas DataFrames

## Install edgartools

In [ ]:
!pip install -U edgartools

## Setup

The SEC requires all automated tools to identify themselves. Replace the email below with your own -- any valid email works.

In [ ]:
import pandas as pd
from edgar import *

# The SEC requires you to identify yourself (any email works)
set_identity("your.name@example.com")

## Get CEO Compensation in 3 Lines

Every public company files a DEF 14A proxy statement disclosing executive pay. edgartools extracts the structured compensation data automatically:

In [ ]:
proxy = Company("AAPL").get_filings(form="DEF 14A")[0].obj()

print(f"Company:           {proxy.company_name}")
print(f"CEO:               {proxy.peo_name}")
print(f"CEO Total Comp:    ${proxy.peo_total_comp:,.0f}")
print(f"CEO Comp Actually Paid: ${proxy.peo_actually_paid_comp:,.0f}")

## Full Compensation Breakdown

The proxy statement includes compensation for the CEO (PEO) and the average of other Named Executive Officers (NEOs), plus shareholder return data:

In [ ]:
print(f"Company:                  {proxy.company_name}")
print(f"CEO:                      {proxy.peo_name}")
print(f"")
print(f"CEO Total Compensation:   ${proxy.peo_total_comp:>15,.0f}")
print(f"CEO Actually Paid:        ${proxy.peo_actually_paid_comp:>15,.0f}")
print(f"NEO Avg Total Comp:       ${proxy.neo_avg_total_comp:>15,.0f}")
print(f"NEO Avg Actually Paid:    ${proxy.neo_avg_actually_paid_comp:>15,.0f}")
print(f"")
print(f"Total Shareholder Return: {proxy.total_shareholder_return}%")
print(f"Peer Group TSR:           {proxy.peer_group_tsr}%")
print(f"Net Income:               ${proxy.net_income:,.0f}")
print(f"Performance Measure:      {proxy.company_selected_measure}")

## 5-Year Compensation History

The `.executive_compensation` property returns a DataFrame with 5 years of CEO and NEO compensation data:

In [ ]:
ec = proxy.executive_compensation

# Format for display
display_df = ec.copy()
display_df["Year"] = pd.to_datetime(display_df["fiscal_year_end"]).dt.year
for col in ["peo_total_comp", "peo_actually_paid_comp", "neo_avg_total_comp", "neo_avg_actually_paid_comp"]:
    display_df[col] = display_df[col].apply(lambda x: f"${x:,.0f}")

display_df[["Year", "peo_total_comp", "peo_actually_paid_comp", "neo_avg_total_comp"]].rename(
    columns={"peo_total_comp": "CEO Total", "peo_actually_paid_comp": "CEO Actually Paid", "neo_avg_total_comp": "NEO Avg Total"}
)

## Pay vs Performance

The SEC requires companies to show how executive pay aligns with shareholder returns. The `.pay_vs_performance` DataFrame includes compensation alongside TSR and net income:

In [ ]:
pvp = proxy.pay_vs_performance

# Build a readable summary
summary = pd.DataFrame({
    "Year": pd.to_datetime(pvp["fiscal_year_end"]).dt.year,
    "CEO Pay ($M)": (pvp["peo_actually_paid_comp"] / 1e6).round(1),
    "NEO Avg Pay ($M)": (pvp["neo_avg_actually_paid_comp"] / 1e6).round(1),
    "Company TSR %": pvp["total_shareholder_return"],
    "Peer TSR %": pvp["peer_group_tsr"],
    "Net Income ($B)": (pvp["net_income"] / 1e9).round(1),
}).set_index("Year")

summary

## Compare CEO Pay Across Companies

Pull CEO compensation from multiple companies to compare total pay, actual pay, and shareholder returns:

In [ ]:
tickers = ["AAPL", "MSFT", "NVDA", "GOOG"]
rows = []

for ticker in tickers:
    proxy = Company(ticker).get_filings(form="DEF 14A")[0].obj()
    rows.append({
        "Ticker": ticker,
        "CEO": proxy.peo_name or "(not disclosed)",
        "Total Comp ($M)": round(float(proxy.peo_total_comp) / 1e6, 1),
        "Actually Paid ($M)": round(float(proxy.peo_actually_paid_comp) / 1e6, 1),
        "Company TSR %": float(proxy.total_shareholder_return),
    })

pd.DataFrame(rows).set_index("Ticker")

## Company-Selected Performance Measures

Each company discloses the metrics they use to determine executive pay. See what drives compensation at different companies:

In [ ]:
tickers = ["AAPL", "MSFT", "NVDA", "GOOG"]

for ticker in tickers:
    proxy = Company(ticker).get_filings(form="DEF 14A")[0].obj()
    measures = proxy.performance_measures or []
    print(f"{ticker:6s} Key metric: {proxy.company_selected_measure or 'N/A'}")
    if measures:
        print(f"       All measures: {', '.join(measures)}")
    print()

## Why EdgarTools?

EdgarTools is free and open-source. Compare accessing executive compensation data:

**With edgartools (free, no API key):**
```python
proxy = Company("AAPL").get_filings(form="DEF 14A")[0].obj()
print(proxy.peo_name, proxy.peo_total_comp)   # CEO and total comp
proxy.pay_vs_performance                       # 5-year DataFrame
```

**Typical paid API approach ($50+/month, API key required):**
```python
from sec_api import QueryApi, XbrlApi
api = XbrlApi(api_key="YOUR_PAID_API_KEY")
# ... find DEF 14A filing, download XBRL, parse ECD taxonomy tags,
# ... extract compensation amounts, handle missing data manually
```

With edgartools, executive compensation data is extracted from XBRL automatically -- no taxonomy knowledge needed, no API key, no monthly fee.

## Quick Reference

```python
from edgar import *
set_identity("your.name@example.com")

# ── Get proxy statement ──
proxy = Company("AAPL").get_filings(form="DEF 14A")[0].obj()

# ── CEO compensation ──
proxy.peo_name                  # CEO name
proxy.peo_total_comp            # CEO total compensation
proxy.peo_actually_paid_comp    # CEO compensation actually paid

# ── Other executives ──
proxy.neo_avg_total_comp        # NEO average total compensation
proxy.neo_avg_actually_paid_comp # NEO average actually paid

# ── Shareholder returns ──
proxy.total_shareholder_return  # Company TSR %
proxy.peer_group_tsr            # Peer group TSR %
proxy.net_income                # Net income

# ── Performance measures ──
proxy.company_selected_measure  # Primary performance metric
proxy.performance_measures      # All performance measures

# ── DataFrames ──
proxy.executive_compensation    # 5-year compensation history
proxy.pay_vs_performance        # Pay aligned with returns
```

## What's Next

You've learned how to extract executive compensation from SEC proxy statements with Python. Here are related tutorials:

- [Compare Company Financials](https://colab.research.google.com/github/dgunning/edgartools/blob/main/notebooks/compare-company-financials-python.ipynb)
- [Extract Revenue and Earnings from SEC Filings](https://colab.research.google.com/github/dgunning/edgartools/blob/main/notebooks/extract-revenue-earnings-python.ipynb)
- [Track Insider Trading from SEC Form 4](https://colab.research.google.com/github/dgunning/edgartools/blob/main/notebooks/insider-trading-sec-form4-python.ipynb)
- [Download and Parse 10-K Annual Reports](https://colab.research.google.com/github/dgunning/edgartools/blob/main/notebooks/download-10k-annual-report-python.ipynb)

**Resources:**
- [EdgarTools Documentation](https://edgartools.readthedocs.io/)
- [GitHub Repository](https://github.com/dgunning/edgartools)
- [PyPI Package](https://pypi.org/project/edgartools/)

---

## Support EdgarTools

If you found this tutorial helpful, here are a few ways to support the project:

- **Star the repo** -- [github.com/dgunning/edgartools](https://github.com/dgunning/edgartools) -- it helps others discover edgartools
- **Visit edgartools.io** -- [edgartools.io](https://www.edgartools.io/) -- for more tutorials, articles, and updates
- **Report issues** -- found a bug or have a feature idea? [Open an issue](https://github.com/dgunning/edgartools/issues)
- **Share this notebook** -- know someone who works with SEC data? Send them the Colab link

*edgartools is free, open-source, and community-driven. No API key or paid subscription required.*